<a href="https://colab.research.google.com/github/AkshaySingh-Github/Deep-Learning-CNN-model-to-recognize-face/blob/main/FaceRecognization_usingCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning CNN model to recognize face

In [1]:

#IMAGE PRE-PROCESSING for TRAINING and TESTING data
 
# Specifying the folder where images are present
TrainingImagePath='/content/drive/MyDrive/Data/Face Images/Final Training Images'
 
from keras.preprocessing.image import ImageDataGenerator
 
# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
train_datagen = ImageDataGenerator(rescale=1./255
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)
 
# Defining pre-processing transformations on raw images of testing data
test_datagen = ImageDataGenerator(rescale=1./255)
 
# Generating the Training Data
training_set = train_datagen.flow_from_directory(TrainingImagePath, target_size=(64, 64),batch_size=32,class_mode='categorical')
 
 
# Generating the Testing Data
test_set = test_datagen.flow_from_directory(TrainingImagePath,target_size=(64, 64), batch_size=32,class_mode='categorical')
 
# Printing class labels for each face
test_set.class_indices

Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

# Creating a mapping for index and face names

In [5]:
#Creating lookup table for all faces
# class_indices have the numeric tag for each face

TrainClasses=training_set.class_indices
 
# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]= faceName
 
# Saving the face map for future reference
import pickle
with open("FaceResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)
 
# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",ResultMap)
 
# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}

 The Number of output neurons:  16


#Creating the CNN face recognition model

### In the below code snippet, I have created a CNN model with

2 hidden layers of convolution

2 hidden layers of max pooling

1 layer of flattening

1 Hidden ANN layer

1 output layer with 16-neurons (one for each face)

In [6]:
# Create CNN deep learning model
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
 
#Initializing the Convolutional Neural Network
classifier= Sequential()
 
#STEP--1 Convolution
# Adding the first layer of CNN
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))
 
# STEP--2 MAX Pooling
classifier.add(MaxPool2D(pool_size=(2,2)))

# ADDITIONAL LAYER of CONVOLUTION for better accuracy 
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
 
classifier.add(MaxPool2D(pool_size=(2,2)))
 
# STEP--3 FLattening
classifier.add(Flatten())
 
# STEP--4 Fully Connected Neural Network
classifier.add(Dense(64, activation='relu'))
 
classifier.add(Dense(OutputNeurons, activation='softmax'))
 
# Compiling the CNN
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

In [7]:
import time
# Measuring the time taken by the model to train
StartTime=time.time()
 
# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=30,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=10)
 
EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
30/30 [==============================] - 82s 2s/step - loss: 72.1036 - accuracy: 0.0759 - val_loss: 3.7208 - val_accuracy: 0.1025
###### Total Time Taken:  2 Minutes ######


# Testing the CNN classifier on unseen images(Validation)

In [11]:
# Making single predictions
import numpy as np
from keras.preprocessing import image
 
ImagePath='/content/drive/MyDrive/Data/Face Images/Final Testing Images/face1/3face1.jpg'
test_image=image.load_img(ImagePath,target_size=(64, 64))
test_image=image.img_to_array(test_image)
 
test_image=np.expand_dims(test_image,axis=0)
 
result=classifier.predict(test_image,verbose=0)
#print(training_set.class_indices)
 
print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

########################################
Prediction is:  face14
